In [1]:
import torch
import scipy.io

In [2]:
torch.set_default_dtype(torch.double)

In [71]:
data = scipy.io.loadmat('ex3data1.mat')
x, y = torch.tensor(data['X']), torch.tensor(data['y']).T
M = y.shape[1]


# Agrega los 1's en columnas
X = torch.cat((torch.ones(M, 1), x), 1)
N = X.shape[1]
K = len(torch.unique(y))

# Vectorizing regularized logistic regression

In [4]:
sigmoid = lambda x: 1 / (1 + torch.exp(-x))

## Vectorizing the cost function

In [5]:
def costFunctionReg(X, y, M, N, initial_theta, lambda_):
    h = sigmoid(torch.mm(initial_theta, X.T))
    reg_cost = torch.square(initial_theta[:,1:]).sum() * lambda_ / (2 * M)
    reg_grad = torch.tensor([0 if i == 0 else initial_theta[:,i] for i in range(N)]) * lambda_

    J = (-torch.mm(torch.log(h), y.T) - torch.mm(torch.log(1 - h), (1 - y).T))/M + reg_cost
    grad = (torch.mm(X.T, (h - y).T).T + reg_grad)/M

    return J.item(), grad

In [72]:
lambda_test = 3
M_test = 5
N_test = 4
theta_test = torch.tensor([[-2., -1., 1., 2.]])
X_test = torch.cat((torch.ones(1, 5), torch.tensor([[1.,2.,3.,4.,5], [6.,7.,8.,9.,10.], [11.,12.,13.,14.,15.]])/10)).T
y_test = torch.tensor([[1., 0., 1., 0., 1.]])

In [7]:
J, grad = costFunctionReg(X_test, y_test, M_test, N_test, theta_test, lambda_test)
J, grad

(2.5348193961097443, tensor([[ 0.1466, -0.5486,  0.7247,  1.3980]]))

## One-vs-all Classification

In [67]:
iterations = 5000
lambda_ = .1
alpha = .3

In [9]:
def gradientDescent(X, y, M, N, initial_theta, alpha, lambda_, iterations):
    theta = initial_theta.clone()
    for i in range(0, iterations):
        _, grad = costFunctionReg(X, y, M, N, theta, lambda_)
        theta -= alpha * grad
    
    return theta

In [24]:
def oneVsAll(X, y, K):

    theta_0 = torch.zeros(1, N)
    all_theta = []
    
    for k in range(1, K + 1):
        theta = gradientDescent(X, (y == k).type(torch.double), M, N, theta_0, alpha, lambda_, iterations)
        all_theta.append(theta[0].numpy())
        
    return torch.tensor(all_theta)

In [68]:
all_theta = oneVsAll(X, y, K)

In [55]:
def predict(X, initial_theta):
    return sigmoid(torch.mm(initial_theta, X.T)).T

In [69]:
predictions = predict(X, all_theta)

In [66]:
def accuracy(predictions, y):
    selected_y = torch.argmax(predictions, dim=1) + 1
    return (selected_y == y).type(torch.double).mean().item()

In [70]:
accuracy(predictions, y)

0.935